In [1]:
from aiogram import Bot, Dispatcher, executor, types
from aiogram.types import ContentType
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.types.message import ContentTypes
from aiogram.dispatcher import FSMContext
from aiogram.contrib.fsm_storage.memory import MemoryStorage
from sqlalchemy import update

API_TOKEN = "5091011572:AAG4NfkC_zZjcsaAFkwLm4ZXOvhEqyLpQhY"


In [2]:
import asyncio

In [3]:
from food.paths import *
from food.search import *
import pandas as  pd
from food.psql import *
import pytz
timezones = pytz.all_timezones
import requests
from requests.structures import CaseInsensitiveDict
import urllib
from tzwhere import tzwhere

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
def geocode(q):
    geocoding_key = '5d96ac126bcb462cb373297924ab2cb4'
    url = "https://api.geoapify.com/v1/geocode/search?"

    params = {"apiKey":geocoding_key, 
            "text":q}

    resp = requests.get(url + urllib.parse.urlencode(params)).json()
    return  pd.json_normalize(resp['features']).sort_values('properties.rank.importance',ascending = False)[['properties.lat','properties.lon']].iloc[0].to_list()
   

In [6]:
def get_tz(q):
    lat,lon = geocode(q)
    return tzwhere.tzwhere().tzNameAt(lat,lon)

In [7]:
async def async_get_tz(q):
    return get_tz(q)

In [8]:
async def async_search_image(url, env='prod'):
    return search_image(url,env)

In [9]:
async def async_geocode(q):
    return geocode(q)

In [10]:
async def  async_insert_on_conflict(*args, **qwargs):
    return insert_on_conflict(*args, **qwargs)


In [11]:
async def add_sender(message):
    sender = message['from'].to_python()
    sender = pd.DataFrame(sender,index=[0]).drop(columns =['is_bot'])
    await async_insert_on_conflict(sender,'users',unique_cols=['id'])

In [12]:
# Initialize bot and dispatcher
bot = Bot(token=API_TOKEN)
storage = MemoryStorage()
dp = Dispatcher(bot, storage=storage)

dishes_table = Dishes.__table__


In [13]:
ml_version = 0.2
"""0.2 - """

'0.2 - '

In [14]:
class CState(StatesGroup):
    init           = State()
    photo_taken    = State()
    measured       = State()
    set_timezone   = State()
    

In [15]:
@dp.message_handler(commands=['start'])
async def send_welcome(message: types.Message):
    
    await CState.init.set()
    await message.reply("""Counting calories as easy as taking pictures. Just capture everything before you eat it\n
                          Now send a photo of your meal to try""")

In [16]:
@dp.message_handler(commands=['set_timezone'])
async def send_welcome(message: types.Message, state: FSMContext):
    await CState.set_timezone.set()
    await message.reply(f"please search your town to set timezone")


In [17]:
@dp.message_handler(state=CState.set_timezone)
async def set_timezone(message: types.Message, state: FSMContext):
    await types.ChatActions.typing()

    global m 
    global tz
    m = message
    await add_sender(message)
    tz = await async_get_tz(message.text)

    df = pd.DataFrame([[m['from']['id'],'tz',tz,pd.Timestamp.utcnow()]],columns = ['user_id','property','value','timestamp'])
    df.to_sql('user_properties',schema = schema,con = engine,if_exists = 'append',index = False)

    await message.reply(f"your tz is set to {tz}")

In [18]:
@dp.message_handler(content_types=ContentType.PHOTO)
async def process_photo(message: types.Message, state: FSMContext):
    await types.ChatActions.typing()
    
    await add_sender(message)

    photo  = message['photo'][-1]
    await photo.download(reference_images_path/photo['file_id'])
    image_url = await photo.get_url()
    dish = await async_search_image(url=image_url, env='prod')
    
    dish['photo_id'] = photo['file_id']
    dish['user_id'] = sender['id'][0]
    dish['ml_version'] = ml_version 
    dish['timestamp']=pd.Timestamp.utcnow()
    dish.to_sql('dishes',schema = schema,if_exists = 'append',index = False,con=engine)
    
    await CState.photo_taken.set()
    async with state.proxy() as data: data['photo_id'] = photo['file_id']

    btns_text = tuple([(str(p)) for p in range(10,510,10)])

    keyboard_markup = types.ReplyKeyboardMarkup(row_width=2)
    keyboard_markup.add(*(types.KeyboardButton(text) for text in btns_text))


    await message.reply("set weight of the dish you are going to eat", reply_markup=keyboard_markup)


In [19]:
@dp.message_handler(lambda message: message.text.isdigit(), state=CState.photo_taken)
async def measure(message: types.Message, state: FSMContext):

    async with state.proxy() as data:  photo_id = data['photo_id']
    stmt = (
            dishes_table.update()
                        .where(dishes_table.c.photo_id == photo_id)
                        .values(grams=int(message.text))
                        .returning(dishes_table.c.id)
    )
    session.execute(stmt)
    session.commit()

In [20]:
if __name__ == '__main__':
    executor.start_polling(dp)

/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/tzwhere/tzwhere.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.timezoneNamesToPolygons[tzname] = WRAP(polys)


In [ ]:
m['from']['id']

78882798

Polling is stopped.


In [14]:
photo_id = 'AgACAgIAAxkBAAIG8WKoe7peZ5foW_cd-M-R9oTXbqRWAAKcvjEbmhxISQWeTbFDWlEcAQADAgADeQADJAQ'
grams = 20

In [ ]:
int(grams)

In [20]:
dishes_table.columns.keys()

['id',
 'description',
 'energy',
 'protein',
 'carb',
 'fat',
 'score',
 'photo_id',
 'user_id',
 'timestamp',
 'ml_version']

In [56]:
import urllib.parse
import tzwhere

In [87]:
geocode('lisboa')

(-75.094692, 4.5796933)

In [70]:
resp['properties']['lon']

84.881012

In [59]:
resp['lat'],resp['lon']

KeyError: 'lat'

In [ ]:
tz = tzwhere.tzwhere().tzNameAt(message.location.latitude, message.location.longitude)

In [16]:
import requests
from requests.structures import CaseInsensitiveDict

url = f"https://api.geoapify.com/v1/geocode/search?text=38%20Upper%20Montagu%20Street%2C%20Westminster%20W1H%201LJ%2C%20United%20Kingdom&apiKey=5d96ac126bcb462cb373297924ab2cb4"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

resp = requests.get(url, headers=headers)

print(resp.status_code)

200


In [18]:
resp.json()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database License',
     'url': 'https://www.openstreetmap.org/copyright'},
    'name': 'Parkhaus Kö Talstraße',
    'housenumber': '1',
    'street': 'Talstraße',
    'suburb': 'Friedrichstadt',
    'district': 'Stadtbezirk 3',
    'city': 'Dusseldorf',
    'state': 'North Rhine-Westphalia',
    'postcode': '40217',
    'country': 'Germany',
    'country_code': 'de',
    'lon': 6.778318997521949,
    'lat': 51.21710155,
    'distance': 0,
    'result_type': 'amenity',
    'formatted': 'Parkhaus Kö Talstraße, Talstraße 1, 40217 Dusseldorf, Germany',
    'address_line1': 'Parkhaus Kö Talstraße',
    'address_line2': 'Talstraße 1, 40217 Dusseldorf, Germany',
    'category': 'parking.cars',
    'rank': {'popularity': 8.995467104553104},
    'place_id': '515fd9c0a7ff1c1b40599994ccfbc99b4940f0